In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [11]:
symbol = 'A'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 2671824, 'calculationPrice': 'close', 'change': 2.4, 'changePercent': 0.01684, 'close': 144.78, 'closeSource': 'iocialff', 'closeTime': 1683886467516, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': 143.44, 'delayedPriceTime': 1706414324815, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 140.78, 'extendedPriceTime': 1651719958017, 'high': 144.78, 'highSource': '5iuee elin1cyetp mrdda ', 'highTime': 1663120567511, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 140.186, 'iexCloseTime': 1650632867457, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 138.41, 'iexOpenTime': 1679115904461, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1719275114415, 'latestPrice': 140.81, 'latestSource': 'Close', 'latestTime': 'March 18, 2022', 'latestUpdate': 1672109353600, 'latestVolume': 2837765, 'low': 136.17, 'lowSource': 'i  eIeertXi cp

In [13]:
price = data['latestPrice']
market_cap = data['marketCap']

In [6]:
columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe= pd.DataFrame(columns = columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series([
        symbol,
        price, 
        market_cap,
        'N/A'
    ],
    index=columns),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,142.54,43468940560,N/A


In [8]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [14]:
symbols_group = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbols_group))[:50]:
    symbol_strings.append(','.join(symbols_group[i]))
    final_dataframe = pd.DataFrame(columns = columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
       final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'].tolist(), 
                    data[symbol]['quote']['marketCap'].tolist(),
                    'N/A'
                ],
                index=columns),
                ignore_index=True
            )

final_dataframe

AttributeError: 'float' object has no attribute 'tolist'

In [ ]:
portfo---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-61-f665acd2fbce> in <module>
     12         pd.Series([
     13             symbol,
---> 14             data[symbol]['quote']['latestPrice'],
     15             data[symbol]['quote']['marketCap'],
     16             'N/A'

KeyError: 'VIAC'lio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again")
    portfolio_size = input("Enter the value of your portfolio:")
    val= float(portfolio_size)
    

In [ ]:
postion_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy']= math.floor(postion_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
writer.save()